In [13]:
%env PATH_FILE_PRIVATE_KEY_ZEROMQ_CLIENT=/home/adam/Engineering/Docker/smart_broker/private_keys/client.key_secret
#%PATH_FILE_PRIVATE_KEY_ZEROMQ_SERVER=/home/adam/Engineering/Docker/smart_broker/private_keys/server.key_secret
%env PATH_DIR_PUBLIC_KEY_ZEROMQ=/home/adam/Engineering/Docker/smart_broker/public_keys

{
  "dataframe_id": "c094f7ae-7ba1-4f48-8f39-e59984facc1e",
  "data_frame_name": "data_frame_0"
}
{
  "data_model_id": "c094f7ae-7ba1-4f48-8f39-e59984facc1e"
}

env: PATH_FILE_PRIVATE_KEY_ZEROMQ_CLIENT=/home/adam/Engineering/Docker/smart_broker/private_keys/client.key_secret
env: PATH_DIR_PUBLIC_KEY_ZEROMQ=/home/adam/Engineering/Docker/smart_broker/public_keys


# Connect to the SCN

In [14]:
from sail_safe_functions_orchestrator.client_rpc_zero import ClientRPCZero
from sail_safe_functions_orchestrator.service_client_dict import ServiceClientDict

scn_names = ["127.0.0.1"]
list_dataset_id = []
list_dataset_id.append("a892ef90-4f6f-11ed-bdc3-0242ac120002")
# list_dataset_id.append("a89301b0-4f6f-11ed-bdc3-0242ac120002")
# list_dataset_id.append("a89302dc-4f6f-11ed-bdc3-0242ac120002")
from sail_safe_functions_orchestrator import preprocessing

service_client = ServiceClientDict()
for dataset_id, scn_name in zip(list_dataset_id, scn_names):
    service_client.register_client(dataset_id, ClientRPCZero(scn_name, 5010))
    print(f"Connected to SCN serving dataset {scn_name}")

Connected to SCN serving dataset 127.0.0.1


# Data Manipulation
## 1. Bring Data to dataframe state

In [38]:
from sail_safe_functions_orchestrator.data_model.data_model_data_frame import (
    DataModelDataFrame,
)
from sail_safe_functions_orchestrator.data_model.data_model_series import (
    DataModelSeries,
)
from sail_safe_functions_orchestrator.data_model.data_model_tabular import (
    DataModelTabular,
)
from sail_safe_functions_orchestrator.preprocessing import convert
from sail_safe_functions_orchestrator import preprocessing



def get_standard_federated_dataframe():

    dataset_federation_id = "a892f738-4f6f-11ed-bdc3-0242ac120002"
    dataset_federation_name = "r4sep2019_csvv1_20_1"

    dataset_longitudinal = preprocessing.read_dataset_fhirv1(
    service_client, list_dataset_id
    )

    data_frame_name = "data_frame_0"

    data_model_data_frame = DataModelDataFrame(data_frame_name)
    data_model_data_frame.add_data_model_series(
        DataModelSeries.create_numerical(
            series_name="bmi_mean",
            measurement_source_name="Observation:Body Mass Index",
            type_agregator=DataModelSeries.AgregatorIntervalMean,
            unit="kg/m2",
        )
    )
    data_model_data_frame.add_data_model_series(
        DataModelSeries.create_numerical(
            series_name="bmi_first",
            measurement_source_name="Observation:Body Mass Index",
            type_agregator=DataModelSeries.AgregatorIntervalFirstOccurance,
            unit="kg/m2",
        )
    )

    data_model_data_frame.add_data_model_series(
        DataModelSeries.create_numerical(
            series_name="bmi_last",
            measurement_source_name="Observation:Body Mass Index",
            type_agregator=DataModelSeries.AgregatorIntervalLastOccurance,
            unit="kg/m2",
        )
    )
    data_model_tabular = DataModelTabular()
    data_model_tabular.add_data_model_data_frame(data_model_data_frame)

    dataset_tabular_federated = convert.convert_to_dataset_tabular(
        dataset_longitudinal,
        dataset_federation_id,
        dataset_federation_name,
        data_model_tabular,
    )

    data_frame_name = dataset_tabular_federated.list_data_frame_name[0]
    data_frame_federated = dataset_tabular_federated[data_frame_name]

    return data_frame_federated

### 1.3 Pull tabular data for longitudinal with given specification

In [48]:
# data_frame_federated = get_standard_federated_dataframe()
data_frame_federated = preprocessing.drop_missing(
        get_standard_federated_dataframe(), axis=0, how="any", thresh=None, subset=None
    )

In [51]:
data_frame_federated.

{'a892ef90-4f6f-11ed-bdc3-0242ac120002': <sail_safe_functions_orchestrator.reference_data_frame.ReferenceDataFrame at 0x7faed3c78ee0>}

In [7]:
!pip3 install requests

  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached certifi-2022.9.24-py3-none-any.whl (161 kB)
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)


# Statistics Functions API and Function Call
## 1. Mean

In [23]:
import requests

def create_data_model_date_frame(data_frame_name):
    payload = {"data_frame_name": data_frame_name}
    result = requests.post("http://127.0.0.1:8000/data_model_frame", params=payload)
    return result.json()["data_model_id"]
data_model_id = create_data_model_date_frame("data_frame_0")
data_model_id


'e7083b37-79a9-40d5-b944-b9ac2b09fab7'

In [24]:

def get_dataframe_name_lookup_table():
    result = requests.post("http://127.0.0.1:8000/admin/get_dataframe_name_lookup_table", params=None)
    return result.json()["dataframe_name_lookup"]
get_dataframe_name_lookup_table()

{'data_frame_0': 'e7083b37-79a9-40d5-b944-b9ac2b09fab7'}

In [25]:
def data_model_add_series(data_model_id, series_name, source_name):
    payload = {"data_model_id": data_model_id,
                "series_name": series_name,
                "source_name": source_name}
    result = requests.post("http://127.0.0.1:8000/data_model_frame/"+data_model_id+"/model_series", params=payload)
    print(result.json())
    return result.json()["data_model_id"]

data_model_add_series(data_model_id, "bmi_mean", "Observation:Body Mass Index")
data_model_add_series(data_model_id, "bmi_first", "Observation:Body Mass Index")
data_model_add_series(data_model_id, "bmi_last", "Observation:Body Mass Index")


{'data_model_id': 'e7083b37-79a9-40d5-b944-b9ac2b09fab7'}
{'data_model_id': 'e7083b37-79a9-40d5-b944-b9ac2b09fab7'}
{'data_model_id': 'e7083b37-79a9-40d5-b944-b9ac2b09fab7'}


'e7083b37-79a9-40d5-b944-b9ac2b09fab7'

In [26]:
def get_data_model_tabular():
    result = requests.post("http://127.0.0.1:8000/data_frame_tabular", params=None)
    return result.json()["data_frame_tabular_id"]
data_model_tabular_id = get_data_model_tabular()

In [27]:
def data_frame_model_tabular_add_dataframe_model(data_model_id, data_frame_tabular_id):
    payload = {"data_model_id": data_model_id,
                "data_frame_tabular_id": data_frame_tabular_id}
    result = requests.post("http://127.0.0.1:8000/data_frame_tabular_add_dataframe_model/"+data_frame_tabular_id+"/"+data_model_id, params=payload)
    return result.json()["data_frame_tabular_id"]

data_model_tabular_id = data_frame_model_tabular_add_dataframe_model(data_model_id, data_model_tabular_id)

In [28]:
dataset_federation_id = "a892f738-4f6f-11ed-bdc3-0242ac120002"
dataset_federation_name = "r4sep2019_csvv1_20_1"

def dataset_tabular_fhirv1(dataset_federation_id, dataset_federation_name, data_model_tabular_id):
    payload = {"dataset_federation_id": dataset_federation_id,
                "dataset_federation_name": dataset_federation_name,
                "data_model_tabular_id": data_model_tabular_id}
    result = requests.post("http://127.0.0.1:8000/dataset_tabular/fhirv1", params=payload)
    return result.json()["dataset_id"]

tabular_dataset_id = dataset_tabular_fhirv1(dataset_federation_id, dataset_federation_name, data_model_tabular_id)

In [29]:
def data_frame_tabular_select_dataframe_model(data_frame_tabular_id, data_frame_name):
    payload = {"data_frame_tabular_id": data_frame_tabular_id,
                "data_frame_name": data_frame_name}
    result = requests.post("http://127.0.0.1:8000/data_frame_tabular/select_dataframe/"+tabular_dataset_id, params=payload)
    return result.json()["data_frame_id"]
data_frame_id = data_frame_tabular_select_dataframe_model(tabular_dataset_id, "data_frame_0")

In [21]:
def data_frame_select_series(data_frame_id, series_name):
    payload = {"data_frame_id": data_frame_id,
                "series_name": series_name}
    result = requests.post("http://127.0.0.1:8000/data_frame/select_series/"+data_frame_id, params=payload)
    return result.json()["series_id"]
data_frame = data_frame_tabular_select_dataframe(data_frame_id, "data_frame_0")

'2ff4f859-ad44-4300-a532-ec7ceb828b2e'

In [ ]:
def count(tabular_dataset_id, data_frame_name, series_name):
    payload = {"dataset_id": tabular_dataset_id,
                "data_frame_name": data_frame_name,
                "series_name": series_name}
    result = requests.post("http://127.0.0.1:8000/statistics/count/"+dataset_id+"/"+data_frame_name+"/"+series_name, params=payload)
    return result.json()["dataset_id"]

count(tabular_dataset_clean_id, "data_frame_0", "bmi_mean")

In [164]:
tabular_dataset_id

'81b2507f-0cf6-47dd-a823-4e0db5851e20'

In [162]:
# def drop_missing(dataset_id, data_frame_name):
#     payload = {"dataset_id": dataset_id,
#                 "data_frame_name": data_frame_name}
#     result = requests.post("http://127.0.0.1:8000/preprocessing/drop_missing/"+dataset_id+"/"+data_frame_name, params=payload)
#     return result.json()["result_data_frame_id"]
# dataframe_clean_id = drop_missing(tabular_dataset_id, "data_frame_0")

In [163]:
dataframe_clean_id

'30a4ee8b-e817-4f86-ad15-0ed9fd32e7de'

In [155]:
def count(tabular_dataset_id, data_frame_name, series_name):
    payload = {"dataset_id": tabular_dataset_id,
                "data_frame_name": data_frame_name,
                "series_name": series_name}
    result = requests.post("http://127.0.0.1:8000/statistics/count/"+dataset_id+"/"+data_frame_name+"/"+series_name, params=payload)
    return result.json()["dataset_id"]

count(tabular_dataset_clean_id, "data_frame_0", "bmi_mean")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import requests
import json

def Mean(series_uuid):
    payload = {"series_uuid": series_uuid}
    result = requests.get("http://127.0.0.1:8000/mean", params=payload)
    return result.json()['mean_sail']

series = "Random UUID"
Mean(series)


85.03455555555556

## 2. Chisquare

In [ ]:
def ChiSquare(series_uuid_1, series_uuid_2):
    payload = {"series_uuid_1": series_uuid_1,
                "series_uuid_2": series_uuid_2}
    result = requests.get("http://127.0.0.1:8000/chisquare", params=payload)
    return result.json()['chisquare_sail']

series_1 = "Random UUID 2"
series_2 = "Random UUID 2"
ChiSquare(series_1, series_2)


[514800.00000000006, 0.0]

## 3. Kolmogorov Smirnov Test

In [ ]:
def KolmogorovSmirnovTest(series_uuid, type_distribution, type_ranking):
    payload = {"series_uuid": series_uuid,
                "type_distribution": type_distribution,
                "type_ranking": type_ranking}
    result = requests.get("http://127.0.0.1:8000/kolmogorovSmirnovTest", params=payload)
    return result.json()['k_statistic_sail'], result.json()['p_value_sail']

series = "Random UUID"
type_distribution="normalunit" 
type_ranking="unsafe"
KolmogorovSmirnovTest(series, type_distribution, type_ranking)

(0.9988888888888889, 0.0)

## 4. Kurtosis

In [ ]:
def Kurtosis(series_uuid):
    payload = {"series_uuid": series_uuid}
    result = requests.get("http://127.0.0.1:8000/kurtosis", params=payload)
    return result.json()['kurtosis_sail']

series = "Random UUID"
Kurtosis(series)

-1.1775818710089418

## 5. Levene Test

In [ ]:
def LeveneTest(series_uuid_1, series_uuid_2):
    payload = {"series_uuid_1": series_uuid_1,
                "series_uuid_2": series_uuid_2}
    result = requests.get("http://127.0.0.1:8000/leveneTest", params=payload)
    return result.json()['f_statistic_sail'], result.json()['p_value_sail']

series_1 = "Random UUID 2"
series_2 = "Random UUID 2"
LeveneTest(series_1, series_2)


(0.0, 1.0)

## 6. Mann Whitney U Test

In [ ]:
def MannWhitneyUTest(series_uuid_1, series_uuid_2, alternative, type_ranking):
    payload = {"series_uuid_1": series_uuid_1,
                "series_uuid_2": series_uuid_2,
                "alternative": alternative,
                "type_ranking": type_ranking}
    result = requests.get("http://127.0.0.1:8000/mannWhitneyUTest", params=payload)
    return result.json()['w_statistic_sail'], result.json()['p_value_sail']

series_1 = "Random UUID 2"
series_2 = "Random UUID 2"
alternative = "two-sided"
type_ranking = "unsafe"
MannWhitneyUTest(series_1, series_2, alternative, type_ranking)

(405000.0, 1.0)

## 7. Min Max

In [ ]:
def MinMax(series_uuid):
    payload = {"series_uuid": series_uuid}
    result = requests.get("http://127.0.0.1:8000/minMax", params=payload)
    return result.json()['min_sail'], result.json()['max_sail']

series = "Random UUID"
MinMax(series)

(80.009133, 89.99036299999999)

## 8. Paired T Test

In [ ]:
def PairedTTest(series_uuid_1, series_uuid_2, alternative):
    payload = {"series_uuid_1": series_uuid_1,
                "series_uuid_2": series_uuid_2,
                "alternative": alternative}
    result = requests.get("http://127.0.0.1:8000/pairedTTest", params=payload)
    return result.json()['t_statistic_sail'], result.json()['p_value_sail']

series_1 = "Random UUID 2"
series_2 = "Random UUID 2"
alternative = "less"
PairedTTest(series_1, series_2, alternative)

(260.0156214109429, 1.0)

## 9. Pearson

In [ ]:
def Pearson(series_uuid_1, series_uuid_2, alternative):
    payload = {"series_uuid_1": series_uuid_1,
                "series_uuid_2": series_uuid_2,
                "alternative": alternative}
    result = requests.get("http://127.0.0.1:8000/pearson", params=payload)
    return result.json()['pearson_sail'], result.json()['p_value_sail']

series_1 = "Random UUID 1"
series_2 = "Random UUID 2"
alternative = "two-sided"
Pearson(series_1, series_2, alternative)

(1.0, 0.0)

## 10. Skewness

In [ ]:
def Skewness(series_uuid):
    payload = {"series_uuid": series_uuid}
    result = requests.get("http://127.0.0.1:8000/skewness", params=payload)
    return result.json()['skewness_sail']

series = "Random UUID"
Skewness(series)


0.03129197977535138

## 11. Spearman

In [ ]:
def Spearman(series_uuid_1, series_uuid_2, alternative, type_ranking):
    payload = {"series_uuid_1": series_uuid_1,
                "series_uuid_2": series_uuid_2,
                "alternative": alternative,
                "type_ranking": type_ranking}
    result = requests.get("http://127.0.0.1:8000/spearman", params=payload)
    return result.json()['spearman_sail'], result.json()['p_value_sail']

series_1 = "Random UUID 2"
series_2 = "Random UUID 2"
alternative = "two-sided"
type_ranking = "unsafe"
Spearman(series_1, series_2, alternative, type_ranking)

(0.0036171481653660843, 0.913706965860571)

## 12. Student T Test

In [ ]:
def StudentTTest(series_uuid_1, series_uuid_2, alternative):
    payload = {"series_uuid_1": series_uuid_1,
                "series_uuid_2": series_uuid_2,
                "alternative": alternative}
    result = requests.get("http://127.0.0.1:8000/studentTTest", params=payload)
    return result.json()['t_statistic_sail'], result.json()['p_value_sail']

series_1 = "Random UUID 2"
series_2 = "Random UUID 2"
alternative = "less"
StudentTTest(series_1, series_2, alternative)

(0.0, 0.5)

## 13. Variance

In [ ]:
def Variance(series_uuid):
    payload = {"series_uuid": series_uuid}
    result = requests.get("http://127.0.0.1:8000/variance", params=payload)
    return result.json()['variance_sail']

series = "Random UUID"
Variance(series)


8.283936732170213

## 14. Welch T Test

In [ ]:
def WelchTTest(series_uuid_1, series_uuid_2, alternative):
    payload = {"series_uuid_1": series_uuid_1,
                "series_uuid_2": series_uuid_2,
                "alternative": alternative}
    result = requests.get("http://127.0.0.1:8000/welchTTest", params=payload)
    return result.json()['t_statistic_sail'], result.json()['p_value_sail']

series_1 = "Random UUID 2"
series_2 = "Random UUID 2"
alternative = "less"
StudentTTest(series_1, series_2, alternative)

(0.0, 0.5)

## 15. Wilcoxon Signed Rank Test

In [ ]:
def WilcoxonSignedRankTest(series_uuid_1, series_uuid_2, alternative, type_ranking):
    payload = {"series_uuid_1": series_uuid_1,
                "series_uuid_2": series_uuid_2,
                "alternative": alternative,
                "type_ranking": type_ranking}
    result = requests.get("http://127.0.0.1:8000/wilcoxonSignedRankTest", params=payload)
    return result.json()['w_statistic_sail'], result.json()['p_value_sail']

series_1 = "Random UUID 2"
series_2 = "Random UUID 2"
alternative = "two-sided"
type_ranking = "unsafe"
WilcoxonSignedRankTest(series_1, series_2, alternative, type_ranking)

(0.0, 6.772582674446599e-149)